<a href="https://colab.research.google.com/github/denzilsaldanha/monument/blob/master/Cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
from PIL import Image
count = 0
for folder in os.listdir('drive/My Drive/train_sample_images/'):
    for file in os.listdir('drive/My Drive/train_sample_images/'+str(folder)):
        if file.endswith('.jpg') or file.endswith('.mpo'):
            try:
                img = Image.open('drive/My Drive/train_sample_images/'+str(folder)+'/'+str(file)) # open the image file
                img.verify() # verify that it is, in fact an image
            except (IOError, SyntaxError) as e:
                count +=1
                os.remove('drive/My Drive/train_sample_images/'+str(folder)+'/'+str(file))
                print('Bad file: ', str(file))
                
                
print("CHECKED ALL FILES")

/usr/local/lib/python3.6/dist-packages/PIL/JpegImagePlugin.py:775: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn("Image appears to be a malformed MPO file, it will be "


CHECKED ALL FILES


In [0]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras


## WHEN UPLOADING THE DATA SET CREATE A FOLDER CALLED DATASET inside which the training and testing folder must be. Change the names accordingy





# Part 1 - Building the CNN
from keras.callbacks import CSVLogger
import os
import glob
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.callbacks import ModelCheckpoint
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (5,5), input_shape = (224, 224, 3), activation = 'relu')) ## Need to change the input shape

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3,3), activation = 'relu'))  ## Change the feature number of features to be used and the feature detector size
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(128, (3,3), activation = 'relu'))  ## Change the feature number of features to be used and the feature detector size
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 150, activation = 'softmax')) ## CHANGE TO SOFTMAX



epochs_run = 0

##TO DO : Write a for loop to iterate to the latest version of the weights available.
# if os.path.isfile("drive/My Drive/dataset/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#   classifier.load_weights("drive/My Drive/dataset/weights.best.hdf5")

if glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-[0-9][0-9]*'):
  weight_to_be_loaded = max(glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-[0-9][0-9]*'))
  classifier.load_weights(weight_to_be_loaded)
  epochs_run = int(weight_to_be_loaded[43:45])

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])## CHANGE THE LOSS

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.1)

# test_datagen = ImageDataGenerator(rescale = 1./255)

                  
                  ##CHANGE THE CLASS MODE
training_set = train_datagen.flow_from_directory('drive/My Drive/GOOGLE_MONUMENT/train_sample_images',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                subset ='validation')
validation_set = train_datagen.flow_from_directory(
    'drive/My Drive/GOOGLE_MONUMENT/train_sample_images', # same directory as training data
    target_size=(224,224),
    batch_size=128,
    class_mode='categorical',
    subset='validation')

# test_set = test_datagen.flow_from_directory('drive/My Drive/dataset/test_set',
#                                             target_size = (64, 64),
#                                             batch_size = 32,
#                                             class_mode = 'binary')

csv_file = 'drive/My Drive/GOOGLE_MONUMENT/log_starting_from_epoch_'+str(epochs_run)+'.csv'
checkpoint="drive/My Drive/GOOGLE_MONUMENT/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint, monitor='val_acc', verbose=1, mode='max')
csv_logger = CSVLogger(csv_file, append=True, separator=';')

es = EarlyStopping(monitor='val_loss',verbose = 1)
callbacks_list = [checkpoint,csv_logger,es]


epoch_count_to_be_run = 25 
step_size_train=training_set.n//training_set.batch_size
classifier.summary()
val_steps = validation_set.n//validation_set.batch_size
classifier.fit_generator(training_set,
                         steps_per_epoch = step_size_train ,
                         epochs = epoch_count_to_be_run,
                         validation_data = validation_set,
                         validation_steps=val_steps,callbacks =callbacks_list,initial_epoch = epochs_run)


print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
 
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")


# scores = classifier.evaluate(training_set, test_set, verbose=0)